<div align="left">
  <img src="https://raw.githubusercontent.com/hud-evals/hud-sdk/main/docs/logo/hud_logo.svg" alt="HUD" width="100"/>
</div>

[ Running SheetBench on an excel agent ]

```bash
export HUD_API_KEY=your_api_key_here
pip install hud-python
pip install pandas # optional for debugging
```

### 1. Verbose example

In [70]:
import requests
import base64
import pandas as pd  # optional for debugging
from io import BytesIO  # optional for debugging

from hud import gym
from hud.taskset import load_taskset

In [71]:
# Loads the SheetBench-50 taskset and enables partial grading
taskset = await load_taskset("SheetBench-50", metadata={"partial": True})

In [72]:
# Load in the first task for testing
test_task = taskset[1]
prompt = test_task.prompt
print(f"Prompt: {prompt}")

# The setup function for the SheetBench tasks is a direct xlsx download link
download_link = test_task.setup.args[0]  # type: ignore

# Download the xlsx file
input_xlsx_req = requests.get(download_link)
input_xlsx_req.raise_for_status()
input_xlsx_file = input_xlsx_req.content

print(pd.ExcelFile(BytesIO(input_xlsx_file)).sheet_names)
print(
    pd.ExcelFile(BytesIO(input_xlsx_file))
    .parse(pd.ExcelFile(BytesIO(input_xlsx_file)).sheet_names[0])
    .head()
)

Prompt: Sheet REV_QTR lists quarterly revenue from Q1-2022 through Q1-2025. Compute the compound annual growth rate between those two points. Enter the result in ANSWER!A1.
['REV_QTR']
   Quarter  Revenue
0  Q1-2022  4200000
1  Q2-2022  4404653
2  Q3-2022  4746171
3  Q4-2022  5062265
4  Q1-2023  5365661


In [73]:
###
### Your agent loop goes here, using *prompt* and *input_xlsx_file*, returns *output_xlsx_file*
###

# For testing, we'll use the gold solution
gold_solution = requests.get(
    "https://gahludmjcsmszgyufydt.supabase.co/storage/v1/object/public/sheetbench/852a6a9e-7e9f-4563-8298-20e80ee0a66a/Copy%20of%20Derivations_CAGR_WORKBOOK.xlsx"
)
output_xlsx_file = gold_solution.content
print(pd.ExcelFile(BytesIO(output_xlsx_file)).sheet_names)
print(
    pd.ExcelFile(BytesIO(output_xlsx_file))
    .parse(pd.ExcelFile(BytesIO(output_xlsx_file)).sheet_names[1])
    .head()
)

['REV_QTR', 'ANSWER']
Empty DataFrame
Columns: [0.241955862]
Index: []


In [ ]:
# Get the base64 encoded xlsx file
base64_output_xlsx_file = base64.b64encode(output_xlsx_file).decode("utf-8")

# Adapt the task to set up an evaluation environment with the output xlsx file
test_task.setup = ("sheets_from_bytes", base64_output_xlsx_file)
test_task.id = None

# Use a hud environment to evaluate the agent
env = await gym.make(test_task)
result = await env.evaluate()
print(f"Reward: {result['reward']}")

# obs, _ = await env.reset() # get obs["screenshot"] to visualize
# await env.stream() # to see the live state of the environment for debugging

Reward: 1.0


In [75]:
# Close the environment
await env.close()

### 2. Running at scale

In [64]:
# Optionally turn off hud logging
# import logging
# logging.getLogger("hud").setLevel(logging.WARNING)

In [69]:
async def run_single_task(task):
    prompt = task.prompt
    input_xlsx_file = requests.get(task.setup.args[0]).content

    # TODO: Implement the agent loop using *prompt* and *input_xlsx_file*
    # TODO: Return the *output_xlsx_file* as a base64 encoded string

    # The input alone will return a 0 reward
    base64_output_xlsx_file = base64.b64encode(input_xlsx_file).decode("utf-8")

    # Run evaluation
    task.setup = ("sheets_from_bytes", base64_output_xlsx_file)
    task.id = None
    env = await gym.make(task)
    result = await env.evaluate()
    await env.close()

    return result["reward"]

In [68]:
# Loading and evaluating 50 tasks should take around 2 minutes, without the agent loop
import asyncio

taskset = await load_taskset("SheetBench-50", metadata={"partial": True})

task_jobs = [run_single_task(task) for task in taskset]
rewards = await asyncio.gather(*task_jobs)

print(f"Average reward: {sum(rewards) / len(rewards)}")

Average reward: 0.0
